# Development of Labelling Functions and Model Training

In this notebook, we present the workflow of training a weakly supervised gene tagger for German medical text using the skweak framework. In particular, we:

1. implement different labelling functions for detecting gene names
2. aggregate their predictions using an Hidden Markov Model (Label Model)
3. train a spaCy NER model on this data
4. evaluate the results on a small set of gold-standard labels

In [1]:
import re
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm

import nltk
from nltk.corpus import stopwords

import spacy
from spacy.tokens import Span, DocBin
from spacy_transformers import Transformer
from spacy_transformers.pipeline_component import DEFAULT_CONFIG

from skweak import heuristics, gazetteers, generative, utils, base
from skweak.base import SpanAnnotator
from skweak.heuristics import SpanEditorAnnotator, VicinityAnnotator, SpanConstraintAnnotator
from skweak.analysis import LFAnalysis

import sklearn.metrics

from evaluation import evaluate, get_results, compute_raw_numbers, _get_probs, show_errors


In [2]:
stops = set(stopwords.words('german'))
random_seed = 42

# Load Data


Here we customize the spaCy tokenizer without statistical model and loading the standard German spaCy model.

In [3]:
nlp = spacy.load('de_core_news_md')
infixes = nlp.Defaults.infixes + [r'([-])']
nlp.tokenizer.infix_finditer = spacy.util.compile_infix_regex(infixes).finditer
nlp.add_pipe('sentencizer')

In the following, we construct a dataframe from all sentences in GGPONC and concatenate them. 

In [4]:
GGPONC_PATH = 'data/ggponc/plain_text/sentences/all_files_sentences/'

In [5]:
sentences = list(Path(GGPONC_PATH).glob('*.txt'))
len(sentences)

10193

In [6]:
def get_df(file):
    df = pd.read_csv(file, delimiter='\t', names =['text'] )
    df['file'] = file.stem
    df['sentence_id'] = df.index
    return df

In [7]:
dfs = [get_df(file) for file in sentences]

In [8]:
sentence_df = pd.concat(dfs)
len(sentence_df)

85996

# Labeling Functions

The Clinical Interpretation of Variants in Cancer (CIViC) database is an opensource, open-access knowledgebase curated by experts on therapeutic, prognostic, diagnostic and predisposing relevance of inherited and somatic variants of every type. Both, genes and variants, are used in this project.

In [9]:
df = pd.read_csv('data/molecular/nightly-GeneSummaries.tsv', sep='\t')
CIVIC_genes = df['name'].tolist()
CIVIC_genes_lower = [c.lower() for c in CIVIC_genes]

In [10]:
df = pd.read_csv('data/molecular/nightly-VariantSummaries.tsv', sep='\t', error_bad_lines=False )
CIVIC_variants = df['variant'].tolist()
CIVIC_variants_lower = [c.lower() for c in CIVIC_variants]

Skipping line 13: expected 29 fields, saw 33
Skipping line 17: expected 29 fields, saw 30
Skipping line 31: expected 29 fields, saw 30
Skipping line 441: expected 29 fields, saw 30
Skipping line 502: expected 29 fields, saw 30
Skipping line 553: expected 29 fields, saw 31



"cue_civic" is based on the CIViC database. If a token contains a gene which is listed in the database, this and the next token are labelled a gene. This way, we do not restrict the function to a 100 percent match but leave it some leeway.

In [11]:
def civic(doc):
    for tok in doc:
        for cue in CIVIC_genes:
            if tok.text.find(cue) == -1:
                continue
            else:
                yield tok.i, tok.i+1, "Gene or Protein"
cue_civic = heuristics.FunctionAnnotator("cue_civic", civic)

The Online Mendelian Inheritance in Man (OMIM) database is the encyclopedic collection of the human medical branch of genetics.

In [12]:
entrez_df = pd.read_csv('Homo_sapiens.gene_info', sep='\t')

In [13]:
symbols = set()
# Get all synonyms for CIViC genes, remove short ones and German stopwords
for _, r in entrez_df.set_index('GeneID').loc[df.entrez_id].iterrows():
    symbols.add(r.Symbol)
    for s in r.Synonyms.split('|'):
        if not s in ['R1', 'R2', 'eN', 'HNPCC'] and len(s) > 1 and not s.lower() in stops:
            symbols.add(s.lower())

In [14]:
from spacy.matcher import Matcher

entrez_matcher = Matcher(nlp.vocab)
pattern = []
for s in nlp.pipe(tqdm(symbols), disable=["ner", "tok2vec"]):
    for pos in ['NOUN', 'PROPN', 'X']: # Consider only if first POS is one of these
        p = [{'LOWER' : spl.text.lower() } for spl in s]
        p[0]['POS'] = pos
        pattern.append(p)
        p2 = p + [{'LOWER' : '-'}, {'LOWER' : 'gen'}] #also consider if followed by -Gen
        pattern.append(p2)
entrez_matcher.add("entrez", pattern)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 2028/2028 [00:00<00:00, 12322.11it/s]


In [15]:
def lf_entrez(doc):
    matches = entrez_matcher(doc)
    if matches:
        # Keep longest matches only
        spans = [doc[start:end] for _, start, end in matches]
        spans = spacy.util.filter_spans(spans)
        for s in spans:
            yield s.start, s.end, 'Gene or Protein'
entrez = heuristics.FunctionAnnotator("entrez", lf_entrez)  

In [16]:
omim_list = pd.read_csv("data/molecular/mim2gene.csv")
omim_list = omim_list['name'].tolist()
omim_list_lower = [o.lower() for o in omim_list]
short_genes = []
for u in omim_list:
    if len(u)<3:
        short_genes.append(u)
less_short_genes = []
for u in omim_list:
    if len(u)<5 and len(u)>2:
        less_short_genes.append(u)
print(len(omim_list))

16767


"omim" is based on the OMIM database and checks whether tokens are present in its list of 16,767 approved gene symbols in lowercase as the diversity of genes often shows in volatile capitalization. To increase precision, genes with a length shorter than three characters are matched only correctly cased.

In [17]:
from skweak.gazetteers import Trie, GazetteerAnnotator

terms = [t.strip() for t in open('proteins.txt', 'r').readlines()]

trie = Trie()
for term in terms:
    trie.add([t.text for t in nlp(term)])

protein_gazetteer = GazetteerAnnotator('protein_gazetteer', tries = {'Gene or Protein' : trie })

doc = nlp("PD-L1")
protein_gazetteer(doc)
doc.spans

{'protein_gazetteer': [PD-L1]}

In [18]:
protein_matcher = Matcher(nlp.vocab)
patterns = []

for suffix in ['[A-Z]*[Kk]inase[n]?$', '[A-Z]+[rR]ezeptor(en|s)?$', '^(RAS|ras)$']:
    p = [{'TEXT' : { 'REGEX' : suffix}}]
    patterns.append(p)
    for _ in range(0, 3): # Consider also combinations like Rezepter-Tyrosinkinasen
        p = [{'IS_ALPHA' : True}, {'lower' : '-'}] + p
        patterns.append(p)
protein_matcher.add('protein', patterns[-1::-1])

def lf_protein_families(doc):
    matches = protein_matcher(doc)
    if matches:
        # Keep longest matches only
        spans = [doc[start:end] for _, start, end in matches]
        spans = spacy.util.filter_spans(spans)
        for s in spans:
            yield s.start, s.end, 'Gene or Protein'

protein_families = heuristics.FunctionAnnotator("protein_families", lf_protein_families) 

In [19]:
list(lf_protein_families(nlp("RAS k-RAS krass")))

[(0, 1, 'Gene or Protein'), (1, 4, 'Gene or Protein')]

In [20]:
def omim(doc):
    for tok in doc:
        if tok.text.lower() in omim_list_lower and tok.text.lower() not in stops and len(tok.text.lower())>=3:
            yield tok.i, tok.i+1, "Gene or Protein"
omim = heuristics.FunctionAnnotator("omim", omim) 

The Catalogue of Somatic Mutations in Cancer (COSMIC) database harbors somatic cell mutations and additional information associated with cancer in humans.

In [21]:
cosmic_census = pd.read_csv("data/molecular/cancer_gene_census.csv")
cosmic_census = cosmic_census['Gene Symbol'].tolist()
cosmic_census_lower = [c.lower() for c in cosmic_census]

"cue_cosmic_census" is based on the COSMIC database. If a token contains a gene symbol which is listed here, this token and its successor are annotated as a gene.

In [22]:
def cosmic(doc):
    for tok in doc:
        for cue in cosmic_census:
            if tok.text.find(cue) == -1:
                continue
            else:
                yield tok.i, tok.i+1, "Gene or Protein"
cue_cosmic_census = heuristics.FunctionAnnotator("cue_cosmic_census", cosmic)  

"construct" is based on the Human Genome Organization (HUGO) Gene Nomenclature Committee (HGNC) naming conventions for genes and leverages regular expressions to let the annotator abide by them. Those expressions comprise various combinations of letters and numbers and certain fixed terms for shorter terms to avoid underfitting. In addition, the CIViC database for variants has also been included for a better recall.

In [23]:
def structure(doc):
    for tok in doc:
        if re.search(r"[a-zA-Z]{4}\d{2}", tok.text) or re.search(r"[a-zA-Z]{5}\d{1}", tok.text)\
        or re.search(r"[a-zA-Z]{4}\d{1}", tok.text) or re.search(r"[A-Z]{5}\d{1}", tok.text)\
        or re.search(r"[A-Z]{5}\d{2}", tok.text) or re.search(r"[A-Z]{3}\d{2}", tok.text)\
        or re.search(r"[a-zA-Z]{2}\d{3}[a-zA-Z]{2}", tok.text) or re.search(r"[a-zA-Z]{1}\d{3}[a-zA-Z]{1}", tok.text)\
        or re.search(r"[A-Z]{3}\d{2}", tok.text) or re.search(r"[A-Z]{6}\d{1}", tok.text)\
        or re.search(r"[A-Z]{3}\d{3}", tok.text) or re.search(r"[p]\d{2}", tok.text)\
        or re.search(r"CYP[a-zA-Z0-9]{3}", tok.text) or re.search(r"CYP[a-zA-Z0-9]{2}", tok.text)\
        or re.search(r"[A-Z]{3}\d{1}", tok.text) or re.search(r"[A-Z]{2}\d{2}", tok.text)\
        or re.search(r"^CK.", tok.text) or re.search(r"^PD-..", tok.text) or re.search(r"^PS[MA|A]", tok.text) or tok.text.lower in CIVIC_variants_lower:
            yield tok.i, tok.i+1, "Gene or Protein"
construct = heuristics.FunctionAnnotator("construct", structure)

In [24]:
# Calculate data stats (number of genes, sentences, etc...) -> Table for Materials
# Intersections of the gazetteers are calculated below.
intersection_db = list(set(cosmic_census_lower) & set(CIVIC_genes_lower) & set(omim_list_lower))
union_db = set(set(cosmic_census_lower).union(set(CIVIC_genes_lower)).union(set(omim_list_lower)))
print(len(intersection_db))
print(len(union_db))

218
16782


We define a list of labelling functions which we want to deploy onto our textual input. Every row in our dataframe is subject to examination. Those listed here was the best performing allocation from ca. 30 labelling functions.

In [25]:
lfs = [construct, cue_civic, omim, cue_cosmic_census, entrez, protein_gazetteer, protein_families]

#For Quick Run with Random Sentences!
#random_files = files_df.sample(n = 10000)
all_docs = []

for sentence_idx, doc in zip(tqdm(list(sentence_df.reset_index().iterrows())), nlp.pipe(sentence_df.text, disable=["ner"])):
    i, row = sentence_idx
    for lf in lfs:
        doc = lf(doc)
    all_docs.append(doc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 85996/85996 [20:17<00:00, 70.62it/s]


Remove files which have been manually annotated from the training dataset

In [26]:
with open('data/molecular/annotated_sentences.txt') as fh:
    annotated_sentences = [l.strip() for l in fh.readlines()]
len(annotated_sentences)

2000

In [27]:
docs, s_index = zip(*[(d, si) for d, si in zip(all_docs, sentence_df.reset_index().index) if not d.text in annotated_sentences])
docs = list(docs)
filtered_sentence_df = sentence_df.reset_index().loc[list(s_index)]
len(docs), len(filtered_sentence_df)

(83624, 83624)

## Training Set Evaluation

In [28]:
from skweak.analysis import LFAnalysis

lfa = LFAnalysis(docs, ['Gene or Protein'])
cov = lfa.lf_coverages().rename(index={'Gene or Protein' : 'Coverage'})
overlap = lfa.lf_overlaps().rename(index={'Gene or Protein' : 'Overlaps'})
pd.concat([cov, overlap])

,cue_cosmic_census,cue_civic,construct,omim,entrez,protein_families,protein_gazetteer
Coverage,0.223103,0.209893,0.365478,0.344048,0.405695,0.017907,0.136651
Overlaps,0.929605,0.980420,0.381526,0.499573,0.685962,0.368852,0.699248


# Label Aggregation

In [29]:
# Training of HMM and Majority Voter
#voter = skweak.voting.SequentialMajorityVoter("maj_voter", labels=["Gen"])
#voter.fit(docs)
hmm = generative.HMM("hmm", ["Gene or Protein"])
hmm.fit(docs)

Starting iteration 1
Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Finished E-step with 4624 documents
Starting iteration 2


         1      -33073.4892             +nan


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Finished E-step with 4624 documents
Starting iteration 3


         2      -32400.1132        +673.3760


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Finished E-step with 4624 documents
Starting iteration 4


         3      -32386.9774         +13.1358


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Finished E-step with 4624 documents


         4      -32383.9291          +3.0482


In [30]:
for d in docs:
    d = hmm(d)
    d.ents = d.spans["hmm"]

Consider subset of files where at least on LF has matched

In [31]:
lf_match = []
for d in docs:
    lf_match.append(any([len(v) > 0 for s, v in d.spans.items() if s != 'hmm']))

filtered_sentence_df['lf_match'] = lf_match
by_file = filtered_sentence_df.groupby('file')['lf_match'].max()
match_files = by_file[by_file.values].index

filtered_docs = [d for d, match in zip(docs, filtered_sentence_df.file.isin(match_files)) if match]
gene_docs = [d for d, match in zip(docs, lf_match) if match]

In [32]:
len(docs), len(filtered_docs), len(gene_docs)

(83624, 35501, 4624)

# Labeling Function Analysis

In [33]:
gold_docs_dev = list(DocBin().from_disk('data/molecular/gold_dev.spacy').get_docs(nlp.vocab))

Our labeling functions must also be deployed onto the gold standard data to evaluate strong supervision against weak supervision.

In [34]:
def apply_hmm(gold_docs):
    for g in tqdm(gold_docs):
        if 'Gene or Protein' in g.spans:
            del g.spans['Gene or Protein']
        for lf in lfs:
            g = lf(g)
        g = hmm(g)

In [35]:
apply_hmm(gold_docs_dev)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:15<00:00, 64.68it/s]


In [36]:
from skweak.utils import display_entities

for d in gold_docs_dev:
    for g in d.spans['hmm']:
        if not g in d.ents:
            display_entities(d, layer='hmm')
            display_entities(d)
            print('------')
            continue

------


------


------


------


------


...

------


------


------


------


------


In [37]:
# HMM / LFs vs. Gold-Standard
#evaluate(gold_docs, ['Gen'], ['lf15', 'hmm'])
evaluate(gold_docs_dev, ['Gene or Protein'], [l.name for l in lfs] + ['hmm'])

tok_precision  tok_recall  \
label           proportion model                                          
Gene or Protein 100.0 %    construct                  0.879       0.244   
                           cue_civic                  0.979       0.366   
                           cue_cosmic_census          0.960       0.342   
                           entrez                     0.937       0.450   
                           hmm                        0.899       0.596   
                           omim                       0.955       0.411   
                           protein_families           1.000       0.067   
                           protein_gazetteer          1.000       0.117   
macro                      construct                  0.879       0.244   
                           cue_civic                  0.979       0.366   
                           cue_cosmic_census          0.960       0.342   
                           entrez                     0.937       0.450   
                           hmm                        0.899       0.596   
                           omim                       0.955       0.411   
                           protein_families           1.000       0.067   
                           protein_gazetteer          1.000       0.117   
micro                      construct                  0.879       0.244   
                           cue_civic                  0.979       0.366   
                           cue_cosmic_census          0.960       0.342   
                           entrez                     0.937       0.450   
                           hmm                        0.899       0.596   
                           omim                       0.955       0.411   
                           protein_families           1.000       0.067   
                           protein_gazetteer          1.000       0.117   
weighted                   construct                  0.879       0.244   
                           cue_civic                  0.979       0.366   
                           cue_cosmic_census          0.960       0.342   
                           entrez                     0.937       0.450   
                           hmm                        0.899       0.596   
                           omim                       0.955       0.411   
                           protein_families           1.000       0.067   
                           protein_gazetteer          1.000       0.117   

                                              tok_f1 tok_cee tok_acc coverage  \
label           proportion model                                                
Gene or Protein 100.0 %    construct           0.382                            
                           cue_civic           0.532                            
                           cue_cosmic_census   0.504                            
                           entrez              0.608                            
                           hmm                 0.716                            
                           omim                0.574                            
                           protein_families    0.126                            
                           protein_gazetteer   0.210                            
macro                      construct           0.382                            
                           cue_civic           0.532                            
                           cue_cosmic_census   0.504                            
                           entrez              0.608                            
                           hmm                 0.716                            
                           omim                0.574                            
                           protein_families    0.126                            
                           protein_gazetteer   0.210                            
micro                      construct 

# Training of Transformer-based NER Models

In [ ]:
utils.docbin_writer(docs, f"output/weak_training_lg.spacy")
#utils.docbin_writer(filtered_docs, f"output/weak_training_md.spacy")

Train test splits for strong supervision

In [ ]:
from sklearn.model_selection import train_test_split
gold_docs_strong = list(DocBin().from_disk('data/molecular/gold_dev.spacy').get_docs(nlp.vocab))

docs_strong_train, docs_strong_dev = train_test_split(gold_docs_strong, test_size=0.2, random_state=random_seed)
utils.docbin_writer(docs_strong_train, 'output/strong_train.spacy')
utils.docbin_writer(docs_strong_dev, 'output/strong_dev.spacy')

In [ ]:
# Train NER model on weak labels with spaCy
!spacy train config.cfg --paths.train output/weak_training_lg.spacy  --paths.dev data/molecular/gold_dev.spacy --output output/weak_ner_lg --gpu-id 0 --code training.py

In [ ]:
# Train NER model on smaller set of weak labels with spaCy
#!spacy train config.cfg --paths.train output/weak_training_md.spacy  --paths.dev data/molecular/gold_dev.spacy --output output/weak_ner_md --gpu-id 0 --code training.py

In [ ]:
# Baseline: train NER model on strong labels with spaCy
!spacy train config.cfg --paths.train output/strong_train.spacy --paths.dev output/strong_dev.spacy --output output/strong_ner --gpu-id 0 --code training.py


# Evaluation

In [38]:
ner_model_weak = spacy.load('output/weak_ner_lg/model-best/')
ner_model_strong = spacy.load('output/strong_ner/model-best/')

In [39]:
from IPython.display import display, Markdown

def print_metrics(gold_docs, is_test : bool):
    display(Markdown('__Labeling Function / HMM performance__'))
    gold_ents = [d.ents for d in gold_docs]
    
    apply_hmm(gold_docs)
    
    display(evaluate(gold_docs, ['Gene or Protein'], [l.name for l in lfs] + ['hmm']).loc['Gene or Protein'])
        
    display(Markdown('__Weak NER Performance__'))
    for d, gold_ent in zip(gold_docs, tqdm(gold_ents)):
        d.set_ents([])
        d = ner_model_weak(d)
        d.spans['ner_model'] = d.ents
        d.set_ents(gold_ent)
    display(evaluate(gold_docs, ['Gene or Protein'], ['ner_model']).loc['Gene or Protein'])
    
    if is_test:
        display(Markdown('__Strong NER Performance__'))
        for d, gold_ent in zip(gold_docs, tqdm(gold_ents)):
            d.set_ents([])
            d = ner_model_strong(d)
            d.spans['ner_model'] = d.ents
            d.set_ents(gold_ent)
        display(evaluate(gold_docs, ['Gene or Protein'], ['ner_model']).loc['Gene or Protein'])

## Dev Set Evaluation

In [40]:
gold_docs_dev_eval = list(DocBin().from_disk('data/molecular/gold_dev.spacy').get_docs(nlp.vocab))

print_metrics(gold_docs_dev_eval, is_test=False)

__Labeling Function / HMM performance__

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:15<00:00, 64.20it/s]


tok_precision  tok_recall  tok_f1 tok_cee  \
proportion model                                                          
100.0 %    construct                  0.879       0.244   0.382           
           cue_civic                  0.979       0.366   0.532           
           cue_cosmic_census          0.960       0.342   0.504           
           entrez                     0.937       0.450   0.608           
           hmm                        0.899       0.596   0.716           
           omim                       0.955       0.411   0.574           
           protein_families           1.000       0.067   0.126           
           protein_gazetteer          1.000       0.117   0.210           

                             tok_acc coverage  ent_precision  ent_recall  \
proportion model                                                           
100.0 %    construct                                   0.833       0.305   
           cue_civic                                   0.944       0.465   
           cue_cosmic_census                           0.928       0.436   
           entrez                                      0.902       0.503   
           hmm                                         0.841       0.680   
           omim                                        0.926       0.524   
           protein_families                            1.000       0.076   
           protein_gazetteer                           0.934       0.120   

                              ent_f1  
proportion model                      
100.0 %    construct           0.446  
           cue_civic           0.624  
           cue_cosmic_census   0.594  
           entrez              0.646  
           hmm                 0.752  
           omim                0.670  
           protein_families    0.142  
           protein_gazetteer   0.212

__Weak NER Performance__

100%|███████████████████████████████████████████████████████████████████████████████████████████████████▉| 999/1000 [00:34<00:00, 28.62it/s]


,,tok_precision,tok_recall,tok_f1,tok_cee,tok_acc,coverage,ent_precision,ent_recall,ent_f1
proportion,model,,,,,,,,,
100.0 %,ner_model,0.903,0.637,0.748,,,,0.855,0.745,0.796


## Test Set Evaluation

In [41]:
gold_docs_test_eval = list(DocBin().from_disk('data/molecular/gold_test.spacy').get_docs(nlp.vocab))

print_metrics(gold_docs_test_eval, is_test=True)

__Labeling Function / HMM performance__

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:13<00:00, 72.69it/s]


tok_precision  tok_recall  tok_f1 tok_cee  \
proportion model                                                          
100.0 %    construct                  0.853       0.190   0.310           
           cue_civic                  0.933       0.350   0.510           
           cue_cosmic_census          0.927       0.342   0.500           
           entrez                     0.951       0.525   0.676           
           hmm                        0.864       0.596   0.706           
           omim                       0.904       0.363   0.518           
           protein_families           0.538       0.027   0.052           
           protein_gazetteer          1.000       0.131   0.232           

                             tok_acc coverage  ent_precision  ent_recall  \
proportion model                                                           
100.0 %    construct                                   0.836       0.280   
           cue_civic                                   0.841       0.473   
           cue_cosmic_census                           0.854       0.473   
           entrez                                      0.890       0.608   
           hmm                                         0.789       0.689   
           omim                                        0.818       0.493   
           protein_families                            0.250       0.012   
           protein_gazetteer                           0.975       0.112   

                              ent_f1  
proportion model                      
100.0 %    construct           0.420  
           cue_civic           0.606  
           cue_cosmic_census   0.608  
           entrez              0.722  
           hmm                 0.736  
           omim                0.616  
           protein_families    0.022  
           protein_gazetteer   0.200

__Weak NER Performance__

100%|███████████████████████████████████████████████████████████████████████████████████████████████████▉| 999/1000 [00:33<00:00, 30.08it/s]


,,tok_precision,tok_recall,tok_f1,tok_cee,tok_acc,coverage,ent_precision,ent_recall,ent_f1
proportion,model,,,,,,,,,
100.0 %,ner_model,0.887,0.621,0.73,,,,0.799,0.723,0.76


__Strong NER Performance__

100%|███████████████████████████████████████████████████████████████████████████████████████████████████▉| 999/1000 [00:33<00:00, 29.84it/s]


,,tok_precision,tok_recall,tok_f1,tok_cee,tok_acc,coverage,ent_precision,ent_recall,ent_f1
proportion,model,,,,,,,,,
100.0 %,ner_model,0.544,0.694,0.61,,,,0.558,0.758,0.642


## Descriptive Statistics

In [42]:
def get_genes(doc_list):
    return [e.text for d in doc_list for e in d.ents if e.label_ == 'Gene or Protein']

In [43]:
len(docs), len(get_genes(docs))

(83624, 5617)

In [44]:
len(gold_docs_dev_eval), len(get_genes(gold_docs_dev_eval))

(1000, 475)

In [45]:
len(gold_docs_test_eval), len(get_genes(gold_docs_test_eval))

(1000, 347)